In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pystan
import datetime
import arviz
import seaborn as sns
import scipy
import sklearn.metrics
import pickle

sns.set()

In [2]:
# dates_to_test = ['2020-09-07', '2020-09-14', '2020-09-21',
#        '2020-09-28', '2020-10-05', '2020-10-12', '2020-10-19',
#        '2020-10-26', '2020-11-02', '2020-11-10', '2020-11-16',
#        '2020-11-23', '2020-11-30', '2020-12-07', '2020-12-14',
#        '2020-12-21']

In [38]:
dates_to_test = ['2020-10-05', '2020-10-12', '2020-10-19',
       '2020-10-26', '2020-11-02', '2020-11-10', '2020-11-16',
       '2020-11-23', '2020-11-30', '2020-12-07', '2020-12-14',
       '2020-12-21', '2020-12-28']

In [4]:
max_D = 10 # maximum delay in the model

In [5]:
data = pd.read_csv('data/df_SIVEP_nowcast_allStates_08-02-2021.csv')
def get_state_data(df, state):
    df_state = df.copy()
    if state == 'Brazil':
        df_state.drop(columns=['State'], inplace=True)
        columns = list(df_state.columns)
        columns.remove('Deaths')
        df_state = df_state.groupby(columns, as_index=False)['Deaths'].sum()
        return df_state
    return df_state[df_state['State'] == state]

data = get_state_data(data, 'Brazil')
data

,Date,Release,Date_index,Release_index,Deaths
0,2019-12-31,2020-07-07,-1,0,1
1,2019-12-31,2020-07-14,-1,7,1
2,2019-12-31,2020-07-21,-1,14,1
3,2019-12-31,2020-07-29,-1,22,1
4,2019-12-31,2020-08-03,-1,27,1
...,...,...,...,...,...
9446,2021-02-03,2021-02-08,399,216,621
9447,2021-02-04,2021-02-08,400,216,486
9448,2021-02-05,2021-02-08,401,216,353
9449,2021-02-06,2021-02-08,402,216,225


In [6]:
# Load the data
data_new = data.copy()
print("First date:", data_new.Date.values[0])
print("Last date:", data_new.Date.values[-1])
data['Release_index'] = data.Release.astype('category').cat.codes

# Filter the data
first_date = '2020-06-30'  # best not to include the first releases
data = data[data['Date'] >= first_date] # cut off early days as they are less relevant
data = data[data['Release'] >= first_date] # cut off early days as they are less relevant

data.head()

First date: 2019-12-31
Last date: 2021-02-07


,Date,Release,Date_index,Release_index,Deaths
5760,2020-06-30,2020-07-07,181,0,722
5761,2020-06-30,2020-07-14,181,1,933
5762,2020-06-30,2020-07-21,181,2,1025
5763,2020-06-30,2020-07-29,181,3,1091
5764,2020-06-30,2020-08-03,181,4,1116


In [7]:
data.Release.unique()

array(['2020-07-07', '2020-07-14', '2020-07-21', '2020-07-29',
       '2020-08-03', '2020-08-10', '2020-08-17', '2020-08-24',
       '2020-08-31', '2020-09-07', '2020-09-14', '2020-09-21',
       '2020-09-28', '2020-10-05', '2020-10-12', '2020-10-19',
       '2020-10-26', '2020-11-02', '2020-11-10', '2020-11-16',
       '2020-11-23', '2020-11-30', '2020-12-07', '2020-12-14',
       '2020-12-21', '2020-12-28', '2021-01-04', '2021-01-11',
       '2021-01-18', '2021-01-25', '2021-02-01', '2021-02-08'],
      dtype=object)

In [8]:
# Utility functions for preparing the data

def sum_two_values(val1, val2):
    """Sums two values, treats NaN as 0 if one value is NaN, returns NaN if both values are NaN.
    Used for summing vectors of deaths."""
    if np.isnan(val1) & np.isnan(val2):
        return np.nan
    if np.isnan(val2):
        return val1
    return val1 + val2

def sum_two_vectors(vec1, vec2):
    """Used for getting a total number of deaths per day reported."""
    vec1 = vec1.values
    vec2 = vec2.values
    # if both values are NaN, sum should give NaN
    # if only one is NaN, this NaN should be treated as 0
    for i in range(len(vec1)):
        vec1[i] = sum_two_values(vec1[i], vec2[i])
    return vec1

def get_weeks_vector(n):
    week_vec = []
    week=0
    counter=0
    for i in range(n):
        week_vec.append(week)
        counter += 1
        if counter == 7:
            counter=0
            week = week+1
    return week_vec

def create_reporting_triangle(raw_data):
    """Creates a reporting triangle from SIVEP data binned by date and release"""
    # create and empty Dataframe to store the reporting triangle
    index = raw_data.Date.unique()  # all Dates in the dataset
    columns = np.sort(raw_data.Release_index.unique()) # maximum number of weeks (releases)
    triangle = pd.DataFrame(index = index, columns = columns)
    
    # fill in the delays info
    for i in triangle.index:  # iterate over all dates
        current_date = raw_data[raw_data['Date'] == i]  # get reported deaths for this date for every release
        all_deaths = current_date.Deaths.values
        # take the first number of deaths appearing -- this is 0 delay
        triangle.loc[i,0] = all_deaths[0]
        next_releases = current_date.Release_index.values
        first_release = next_releases[0]
        for j in range(1,len(all_deaths)):
            triangle.loc[i,next_releases[j]-first_release] = all_deaths[j] - all_deaths[j-1]
#     triangle[triangle < 0] = 0
    return triangle

def sum_up_delays_above_max(triangle, max_D = max_D):
    """Sums up all deaths reported with delay larger than max_delay in the reporting triangle"""
    triangle_max_D = triangle.copy()
    max_delays_reported = int(triangle_max_D.columns[-1])
    for c in range(max_D+1, max_delays_reported+1):
        triangle_max_D.loc[:,max_D] = sum_two_vectors(triangle_max_D.loc[:,max_D],
                                                        triangle_max_D.loc[:,c])
        triangle_max_D.drop(columns = [c], inplace = True)
    return triangle_max_D

def mask_bottom_triangle(unmasked_triangle, val=0):
    """Adds a mask to obtain a correct reporting triangle;
    issue arising due to the fact that releases are not in equal weekly intervals"""
    nrow = unmasked_triangle.shape[0]
    ncol = unmasked_triangle.shape[1]
    masked_triangle = unmasked_triangle.copy()
    
    for c in range(0, ncol-1):
        masked_triangle.iloc[nrow-c-1,c+1] = val
    return masked_triangle

def test_reporting_triangle(triangle):
    """Tests whether the reporting triangle has a correct structure.
    Returns True if the structure is correct and False if it is not."""
    test_triangle = triangle.copy()  
    test_triangle.replace(0, np.nan, inplace=True) # might cause issues if 0s are outside of the missing information
    nans_per_row = test_triangle.isnull().sum(axis=1)

    for i in range(1,len(nans_per_row.values)):
        if nans_per_row[i] - nans_per_row[i-1] > 1:
            return False
        if nans_per_row[i] > 0:
            if nans_per_row[i] - nans_per_row[i-1] != 1:
                return False
    return True    

def mask_below_diagonal(triangle):
    """Makes all values below nan value to be nan"""
    nrow = triangle.shape[0]
    ncol = triangle.shape[1]
    masked_triangle = triangle.copy()
    
    for c in range(1, ncol):
        nan_idx = np.argwhere(np.isnan(masked_triangle.iloc[:,c].values))
        if nan_idx:
            for r in range(nan_idx[0][0], nrow):
                masked_triangle.iloc[r,c] = np.nan
    return masked_triangle

def get_results_df(fit, savefile = ''):
    fit_df_GP = fit.extract()['sum_n_predict']
    
    predicted_GP_mean = np.mean(fit_df_GP, axis = 0)
    predicted_GP_median = np.median(fit_df_GP,axis = 0)
    predicted_q025_GP = np.quantile(fit_df_GP,0.025,axis=0)
    predicted_q975_GP = np.quantile(fit_df_GP,0.975,axis=0)
    predicted_q25_GP = np.quantile(fit_df_GP,0.25,axis=0)
    predicted_q75_GP = np.quantile(fit_df_GP,0.75,axis=0)
    
    nowcasted_data = pd.DataFrame(columns = ['week','mean', 'median', 'q025', 'q975', 'q25', 'q75'])
    nowcasted_data['week'] = range(len(predicted_GP_mean))
    nowcasted_data['mean'] = predicted_GP_mean.astype(int)
    nowcasted_data['median'] = predicted_GP_median.astype(int)
    nowcasted_data['q025'] = predicted_q025_GP.flatten().astype(int)
    nowcasted_data['q975'] = predicted_q975_GP.flatten().astype(int)
    nowcasted_data['q25'] = predicted_q25_GP.flatten().astype(int)
    nowcasted_data['q75'] = predicted_q75_GP.flatten().astype(int)

    if savefile:
        nowcasted_data.to_csv(savefile + '.csv', index=False)
    return nowcasted_data

def nowcasting_error(deaths_nowcasted, deaths_true, weeks_test, method):
    """method: 'MSE', 'MAE', 'RMSE'
    weeks_test: how many last weeks use for testing"""
    n = len(deaths_nowcasted)
    deaths_true = deaths_true[:n] # doesn't matter what is later
    deaths_true = deaths_true[-weeks_test:] # we only test last few weeks
    deaths_nowcasted = deaths_nowcasted[-weeks_test:]
    
    if method == 'MSE':
        return sklearn.metrics.mean_squared_error(y_true = deaths_true, y_pred = deaths_nowcasted, squared = True)
    elif method == 'RMSE':
        return sklearn.metrics.mean_squared_error(y_true = deaths_true, y_pred = deaths_nowcasted, squared = False)
    elif method == 'MAE':
        return sklearn.metrics.mean_absolute_error(y_true = deaths_true, y_pred = deaths_nowcasted)
    else:
        print('Wrong method')
        return -1000

    
def nowcasting_error_weighted(deaths_nowcasted, deaths_true, deaths_raw,
                              weeks_test, method):
    """method: 'MSE', 'MAE', 'RMSE'
    weeks_test: how many last weeks use for testing"""
    n = len(deaths_nowcasted)
    deaths_true = deaths_true[:n] # doesn't matter what is later
    deaths_true = deaths_true[-weeks_test:] # we only test last few weeks
    deaths_nowcasted = deaths_nowcasted[-weeks_test:]
    deaths_raw = deaths_raw[-weeks_test:]
    
    # calculate the weights
    weights = abs(deaths_true - deaths_raw)

    if method == 'MSE':
        return sklearn.metrics.mean_squared_error(y_true = deaths_true, 
                                                  y_pred = deaths_nowcasted, 
                                                  sample_weight = weights,
                                                  squared = True)
    elif method == 'RMSE':
        return sklearn.metrics.mean_squared_error(y_true = deaths_true, 
                                                  y_pred = deaths_nowcasted,
                                                  sample_weight = weights,
                                                  squared = False)
    elif method == 'MAE':
        return sklearn.metrics.mean_absolute_error(y_true = deaths_true,
                                                   y_pred = deaths_nowcasted,
                                                   sample_weight = weights)

    else:
        print('Wrong method')
        return -1000

In [9]:
def nowcasting_prep(now_date, maxD=max_D):

    # Prepare the data for nowcasting
    data_filtered = data[data['Release'] <= now_date].copy() # remove the data above nowcasting date

    # 1. Change the Release index to start from 0
    releases = np.sort(data_filtered.Release_index.unique())
    for i in range(len(releases)):
        data_filtered.loc[data_filtered["Release_index"] == releases[i], "Release_index"] = i

    # 2. Create a reporting triangle for the validation data
    delays_data = create_reporting_triangle(data_filtered)

    # 3. Sum up the deaths above max_D (maximum delay to be used)
    delays_data = sum_up_delays_above_max(delays_data, maxD)

    # 4. Bin the data into weeks
    first_date = datetime.datetime.strptime(delays_data.index[0],'%Y-%m-%d')
    last_date = datetime.datetime.strptime(delays_data.index[-1],'%Y-%m-%d')
    delays_data['week'] = get_weeks_vector(len(delays_data.index))

    delays_data_weekly = delays_data.groupby(['week']).sum(min_count=1)

    # 5. Mask some values
    # this is done to keep a correct structure of the reporting triangle
    # this issue is cause because the columns are 'releases' and rows are 'weeks', and the release tend to be weekly
    # with some shorter/longer periods in the beginning of August
    delays_data_weekly = mask_bottom_triangle(delays_data_weekly)

    # 5.5 change the begative values into 0s
    delays_data_weekly[delays_data_weekly < 0] = 0


    # 6. Add a total number of deaths per date reported
    delays_data_weekly["all_deaths"] = delays_data_weekly.sum(axis=1)
    
    triangle = delays_data_weekly.iloc[:,:-1].copy()
    triangle = mask_bottom_triangle(triangle, np.nan)
    triangle = mask_below_diagonal(triangle)
    triangle.replace(np.nan, 10000000, inplace=True)
    triangle = triangle.astype('int32')

    return delays_data_weekly, triangle

In [10]:
delays_data = dict()
triangles = dict()
for d in dates_to_test:
    delays, tri = nowcasting_prep(d)
    delays_data.update({d: delays})
    triangles.update({d: tri})

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [11]:
delays_data['2020-11-10']

,0,1,2,3,4,5,6,7,8,9,10,all_deaths
week,,,,,,,,,,,,
0,3844,2179,810,520,142,268,154,178,132,82,446,8755
1,3793,2457,922,281,329,224,214,153,85,68,432,8958
2,4089,2363,536,581,329,234,212,107,62,80,364,8957
3,4697,1412,918,481,333,234,144,97,55,86,311,8768
4,2392,3250,1065,502,282,183,111,104,95,91,228,8303
5,3326,2519,760,420,169,147,149,98,83,64,194,7929
6,3002,2522,729,318,208,175,120,145,45,58,152,7474
7,2959,2301,534,288,202,165,128,74,65,69,82,6867
8,2913,2136,617,359,212,140,101,67,93,68,50,6756


In [12]:
triangles[dates_to_test[-1]]

,0,1,2,3,4,5,6,7,8,9,10
week,,,,,,,,,,,
0,3844,2179,810,520,142,268,154,178,132,82,564
1,3793,2457,922,281,329,224,214,153,85,68,571
2,4089,2363,536,581,329,234,212,107,62,80,509
3,4697,1412,918,481,333,234,144,97,55,86,461
4,2392,3250,1065,502,282,183,111,104,95,91,358
5,3326,2519,760,420,169,147,149,98,83,64,378
6,3002,2522,729,318,208,175,120,145,45,58,333
7,2959,2301,534,288,202,165,128,74,65,69,333
8,2913,2136,617,359,212,140,101,67,93,68,308


In [13]:
def fit_model(model, triangle):
    T =  triangle.shape[0]
    D = triangle.shape[1]
    data_stan = {'T': T, 
             'D': D, 
             'n': triangle,
            'x': list(range(0,T)),
            'time': list(range(0,T)),
        'delay': list(range(0,D))}
    fit = model.sampling(data=data_stan,iter=1000,#was 1000
                                   warmup=400, #was 400
                                   chains=4,
                                   thin=1, 
                                   n_jobs=-1, 
                                   control=dict(adapt_delta = 0.95, max_treedepth=14), 
                                   seed = 6234+5)   # +5 for 2D sometimes diverges and might need changing
                         
    return fit

In [14]:
def fit_to_all_dates(model, model_str):
    print('Fitting started for', model_str)
    fit_all = {}
    for d in dates_to_test:
        print('Fitting for date', d)
        n = triangles[d]
            
        try:
            fit = fit_model(model, n)
            fit_all.update({d: fit})
            print('Fit finished for date', d)
        except:
            print('Fitting failed for', model_str, 'date', str(d))   
    return fit_all

In [15]:
def save_all_results(fits, model_name):
    for k in fits.keys():
        tmp = get_results_df(fits[k], savefile = model_name + '_' + k)
        
def get_all_results(fits):
    results = {}
    for k in fits.keys():
        tmp = get_results_df(fits[k], savefile = '') # do not save to csv
        results.update({k: tmp})
    return results

In [16]:
def pickle_model(model, fits, savefile):  # 'stan_model_fits/name'
    for d in dates_to_test:
        with open(savefile + '_' + d + ".pkl", "wb") as f:
            pickle.dump({'model' : model, 'fit' : fits[d]}, f, protocol=-1)

## Fiting all models to each of the dates
### Saves the compact results needed for plots and analysis and pickles the model fits
#### Takes a lot of time!

In [34]:
model_kron_2D_additive = pystan.StanModel(file='stan_models/2D_additive.stan')
fits_kron_2D_additive = fit_to_all_dates(model_kron_2D_additive, '2D additive model')


In [33]:
save_all_results(fits_kron_2D_additive, 'tests_results/kron_2D_additive')


In [35]:
results_kron_2D_additive = get_all_results(fits_kron_2D_additive)


In [36]:
pickle_model(model_kron_2D_additive, fits_kron_2D_additive, '../nowcasting/paper_models/backtesting/stan_models_fits/kron_2D_additive')

C:\Users\iwona\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  after removing the cwd from sys.path.


In [39]:
fits_kron_2D_additive.keys()

dict_keys(['2020-10-05', '2020-10-12', '2020-10-19', '2020-10-26', '2020-11-02', '2020-11-10', '2020-11-16', '2020-11-23', '2020-11-30', '2020-12-07', '2020-12-14', '2020-12-21', '2020-12-28'])